# 論文輪読
那須野薫(Kaoru Nasuno)/ 東京大学松尾研究室(Matsuo Lab., the University of Tokyo)

##メタ情報
タイトル：Feedforward Sequential Memory Neural Networks without Recurrent Feedback  
著者：Shiliang Zhang, Hui Jiang, Si Wei, Lirong Dai  
公開：2015年10月  
リンク： [arXiv](http://arxiv.org/abs/1510.02693)  
ページ数：4  



## Abstract
Feedforward Sequential Memory Networks(FSMN)という新しい構造のメモリネットを提案する。  
このメモリネットはrecurrent feedbackを利用しないで長期依存を学習することができる。   
FSMNは隠れ層に学習可能なメモリブロックがついている標準的なFeedforward neural networks(FNN)である。  
この論文では、FSMNをいくつかの言語モデリングのタスクに適用させた。  
実験では、FSMNのメモリブロックは長期履歴の表現を効率的に学習できることが示された。  
また、FSMNに基づいた言語モデルは他のFNNに基づいた言語モデルだけでなく、RNNに基づいた言語モデルも圧倒した。  

### 用語
メモリネット：メモリに相当する機構を持っているNN。  
recurrent feedback：RNNの時系列方向のback propagation。  
長期依存：時系列解析では短期依存の学習は容易。一方で長期依存を学習させることが難しく課題となっている。  
メモリブロック：メモリに相当する機構。  
言語モデリング：文章を学習対象とし、次の単語を予測するというモデリング。  
RNN：言語モデリングの領域でstate of the artのNN。LSTM-RNNが長期依存をよく学習できることが分かっている。  



##選択理由
#### ここずっと時系列解析をやっている。
- Kaggleやプロジェクト。
- RNNはちゃんと学習させるの大変。学習時間とか，その他もろもろ。



#### RNNではなくFNNで系列をよく捉えられる方法をを提案している。
- 紹介する論文は数式の理解と実装が簡単。  
- 精度がちゃんと出ている模様。



## 感想
実装が意外とだるい。  
精度まぁまぁ、引き続き検証。

## 目次
- 背景・イントロ
- FSMN
- 実装
- 実験









## 背景・イントロ
4ページだけなので、細かいことを知りたい方は全部読んでください。  

テキスト、会話、動画などの系列データのモデリングを機械学習で行うとき、長期依存を考慮することがとても大事。  
伝統的なアプローチは、RNNやLSTM-RNNのrecurrent feedbacksで系列の長期依存構造を捉えようとするもの。   
RNNは系列処理で人気。  
特に最近では、RNNではなく明示的なメモリをもつneural computing modelも注目されている。  
ここでは、4つ引用しており(いずれも2014〜)、  
例えば、読み書きできるメモリブロックを持つもの(End-To-End Memory Networks)、   
外部メモリへの接続を行うもの(Neural Turing Machine)がある。  

この論文では、recurrent feedbackを使わないで長期依存を学習できるシンプルなメモリネット(FSMN)を提案している。  
隠れ層にメモリブロックを導入して標準的なFNNを拡張する。  
RNNとは異なり、最終的なFSMNの構造は単純なfeedward structureと同じで、RNNやLSTM-RNNより学習はより効率的に安定して行える。  
FSMNを言語モデリング(LM)の2つのデータセットに適用して、FNN-LMsだけでなく、RNN-LMsも圧倒した。  



## Feedfoward Sequential Memory Networks(FSMN)  
<img src='files/FSMN.png' width="900px"/>
### 数式
$$
{\bf X} = \{{\bf x}_1 , {\bf x}_2 , \ldots , {\bf x}_T\}
$$  
$$
{\bf H} = \{{\bf h}_1, {\bf h}_2, \ldots , {\bf h}_T\}
$$  
$$
{\bf \tilde{h}}_t = f(\sum^{N}_{i=0}a_i \cdot {\bf h}_{t-1})
$$  
$$
{\bf a} = \{a_0 , a_1 , a_2 , \ldots, a_N \}
$$  
$$
{\bf M} = \left[
    \begin{array}{rrrrrrr}
      a_0 & a_1 & \ldots & a_N & 0 & \ldots & 0 &\ldots \\
      0 & a_0 &  a_1 & \ldots & a_N & 0 & \ldots & \\
     &&&&&&& \\
      \vdots & \ldots & 0 &  & \ddots & \ddots & & \vdots  \\
      0 & \ldots & & & a_0 & & \ldots & a_N \\
      &&&&&&& \\
      &&&&&&& \\
      \vdots & & \ldots  &  & \ddots & \ddots & & \vdots  \\
      0  & \ldots & & & & &  0 & a_0
    \end{array}
  \right]
$$  
$$
{\bf \tilde{H}} = f({\bf H} \cdot {\bf M})
$$  

${\bf X}$：入力行列(系列順に並んでいる)  
${\bf H}$：隠れ層行列  
$T$：${\bf X}$の行数(系列数)  
${\bf h}_i$：隠れ層行列の$i$行  
${\bf M}$：メモリを計算するための行列    
${\bf H}$：隠れ層のメモリ行列  

例えば、
$$
{\bf H}_{l+1} = \phi({\bf H}_{l} \cdot {\bf W}_1 + {\bf \tilde{H}}_{l} \cdot {\bf W}_2 + b)
$$  

文章等、$K$個の系列でそれぞれ系列長が異なる場合、
$$
{\bf \tilde{H}}= f([{\bf H_1}, {\bf H_2}, \ldots ,{\bf H_K} ] \cdot
\left[
    \begin{array}{rrrr}
    {\bf M_1} & & &\\
    & {\bf M_2} & & \\
    & & \ddots & \\
    & & & {\bf M_K} \\
    \end{array}
  \right]
)
= f({\bf \bar{H}} \cdot {\bf \bar{M}})
$$  
のようにすればいいと書いてある。  
しかし、実際には、これはいけていないさそうである。  
例えば、平均系列長が$100$で系列数が$100,000$だと$100 \times 100,000$と、  
${\bf \tilde{H}}$は行数が1000万ぐらいになってしまい、計算が厳しそうである。  
バッチごとに行列を作るか用意しておくか、するのが妥当なやりかただろうか。。。  

### 解釈
信号処理の観点では、  
FSMNのメモリブロックは高次有限インパルス応答(high-order finite impuse response(FIR))フィルタとみなせるが、  
RNNのrecurrent layer、つまり$ \tilde{{\bf h}}_t = f({\bf h}_t + {\bf W} \cdot \tilde{{\bf h}}_{t-1})$は一次無限インパルス応答(first-order infinite impulse reponse(IIR))フィルタとみなせる。  
明らかに、${\bf a}はN番目のFIRフィルタの係数とみなせる。  
(一般的に、おそらく信号処理の領域では？)、IIRフィルタはFIRフィルタよりもコンパクトだが、  
IIRフィルタは実装が難しいかもしれない。  
場合によってはIIRフィルタは不安定になるが、FIRは安定している。  
さらに、IIRフィルタのようなRNNの学習には、計算の複雑さや勾配消滅や勾配爆発という問題を抱えているBPTTが必要だが、  
一方で、提案するFIRフィルタのようなFSMNは標準的なBPの手続きで効率的に学習できる。  
したがって、FSMNの学習はRNNより安定していて、効率的である。  


## 実装
theanoで実装した。  
特に実装が面倒だったのが${\bf M}$の部分。  
learnable parameterが一部で、かつ複数部分が同じパラメタなので。


In [1]:
import numpy
import theano
import theano.tensor as T

def relu(x):
    return T.switch(T.gt(x, 0), x, 0.0)

def step_fsmn_M(i, a, n_hists, n_hidden_back):
    #ステップ関数
    n_rep = T.minimum(n_hists-i, n_hidden_back)
    m = T.set_subtensor(T.zeros((n_hists, ), dtype='float32')[i: i+n_rep], a[:n_rep])
    return m

def get_fsmn_M(a, n_hists, n_hidden_back):
    # scanで行ごとに定義・置換していく。
    M, _ = theano.scan(
        fn=step_fsmn_M,
        sequences=T.arange(n_hists),
        outputs_info=None,
        non_sequences=[a, n_hists, n_hidden_back],
    )
    return M

def get_init_weight(rng, n_in, n_out):
    return rng.uniform(
        low=-numpy.sqrt(6. / (n_in + n_out)),
        high=numpy.sqrt(6. / (n_in + n_out)),
        size=(n_in, n_out),
    ).astype('float32')


n_x = 100  # FNNへの入力の次元
n_y = 10  # FNNの出力の次元
n_hidden_mlp = 100  # 隠れ層の次元
n_hidden_back = 10   # 何個前まで考えるか。
n_hists = 1000  #  FNNへの入力の行数

rng = numpy.random.RandomState(1234)  # Random Seed
X = T.fmatrix('X')
Y = T.fmatrix('Y')

params = []


W1 = theano.shared(get_init_weight(rng, n_x, n_hidden_mlp), name='W1')
b1 = theano.shared(numpy.zeros((n_hidden_mlp,), dtype='float32'), name='b1')
fsmn_a1m = theano.shared(numpy.ones((n_hidden_back,), dtype='float32') / n_hidden_back, name='fsmn_a1m')

W2 = theano.shared(get_init_weight(rng, n_hidden_mlp, n_hidden_mlp), name='W2')
W2m = theano.shared(get_init_weight(rng, n_hidden_mlp, n_hidden_mlp), name='W2m')
b2 = theano.shared(numpy.zeros((n_hidden_mlp,), dtype='float32'), name='b2')


W3 = theano.shared(get_init_weight(rng, n_hidden_mlp, n_y), name='W3')
b3 = theano.shared(numpy.zeros((n_y,), dtype='float32'), name='b3')

params += [W1, b1]
params += [fsmn_a1m]
params += [W2, W2m, b2]
params += [W3, b3]

H1 = relu(T.dot(X, W1) + b1)  # 1層目のアフィン変換

M1 = get_fsmn_M(fsmn_a1m, n_hists, n_hidden_back)
H1m = relu(T.dot(H1.T, M1).T)  # 1層目のメモリ

H2 = relu(T.dot(H1, W2) + T.dot(H1m, W2m) + b2)  # 2層目のアフィン変換

Y_pred = relu(T.dot(H2, W3) + b3)  # 最後の予測

cost = T.sum((Y - Y_pred) ** 2)  # cost


## 実験
隠れ層の活性化関数にはReluを、  
初期化はXavier。

<img src='files/FSMN_EXP.png' width="900px"/>

